In [27]:
# compare route outputs from different levels of complexity
import os, datetime
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import multiprocessing as mp
from tqdm import tqdm 

# ========== User settings ==================================
root_dir='/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/calib'
basinName='06279940'

# hru inputs
level_num=3
label_basename='hru_lev'

# calib inputs
calib_run = 1

# hru shapefile
case = 'shoshone'
discretize_dir = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/discretize'
case_dir = os.path.join(discretize_dir,case)
sub_shp_prj = os.path.join(discretize_dir, case, 'subbasin_prj.shp')
stream_clip = os.path.join(discretize_dir, case, 'stream.shp')
wgs_crs = 'epsg:4326'

# plot inputs
time_format = '%Y-%m-%d'
plot_date_start = '2007-10-01' #'2007-10-01'
plot_date_end = '2008-09-30'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)
plot_date_range = pd.date_range(start=plot_date_start,end=plot_date_end,freq='M').to_pydatetime().tolist()

var_names_summa = ['SWRadAtm','LWRadAtm','scalarSWE']
var_names_plot = ['Shortwave Rad','Longwave Rad','SWE']
var_units_plot = ['$(W/m^2)$','$(W/m^2)$','$(mm)$'] 
cmaps=['OrRd','GnBu','YlGn']
            
output_dir=os.path.join(root_dir,basinName,'analysis/c8_plot_swe_rad_month_space')
output_subdir=os.path.join(output_dir,'plot_month')
if not os.path.exists(output_subdir):
    os.makedirs(output_subdir)

# ========== end User settings ==================================

##============== PART 1. Read data ==============
# --- read summa input & output
print('read')
data_dic = {}
var_units_update = []
for level_id in range(level_num+1):
    label=label_basename+str(level_id)
    data_lev_dic = {}
    
    # read SWE from summa output
    q_ncfile = os.path.join(root_dir,basinName,label,'out_archive', 
                            str(calib_run),'wbout_day.nc')
    ds = xr.open_dataset(q_ncfile) 
    time = ds['time'].values[:]
    time = pd.to_datetime(time)
    scalarSWE = ds['scalarSWE'].values[:] # (time, hru). unit: 'mm'    
    df = pd.DataFrame(scalarSWE, index=time)
    
    # calculate monthly aggregate (sum)
    df_sum = df.resample('M').sum()
    data_lev_dic['scalarSWE'] = df_sum
    
    # read radiation from summa input
    plot_start_yr = plot_date_start_obj.year
    plot_end_yr = plot_date_end_obj.year
    for yr in range(plot_start_yr, plot_end_yr+1):
        force_ncfile = os.path.join(root_dir,basinName,'model/forcings/summa_3hr_'+label, 
                        'ens_forc.'+basinName+'.huc12.'+str(yr)+'.001.nc')
        ds = xr.open_dataset(force_ncfile) 
        time = ds['time'].values[:]
        time = pd.to_datetime(time)
        
        SWRadAtm = ds['SWRadAtm'].values[:] # (time, hru). unit: W m-2    
        LWRadAtm = ds['LWRadAtm'].values[:] # (time, hru). unit: W m-2    
        
        if yr == plot_start_yr:
            time_concat = time
            SWRadAtm_concat = SWRadAtm
            LWRadAtm_concat = LWRadAtm
        else:
            time_concat = np.concatenate((time_concat, time), axis=0)
            SWRadAtm_concat = np.concatenate((SWRadAtm_concat, SWRadAtm), axis=0)
            LWRadAtm_concat = np.concatenate((LWRadAtm_concat, LWRadAtm), axis=0)
    
    # calculate monthly mean of 3-hr radiation
    df_SW = pd.DataFrame(SWRadAtm_concat, index=time_concat)
    df_LW = pd.DataFrame(LWRadAtm_concat, index=time_concat)

    df_SW_avg = df_SW.resample('M').mean()
    df_LW_avg = df_LW.resample('M').mean()    
    
    data_lev_dic['SWRadAtm'] = df_SW_avg
    data_lev_dic['LWRadAtm'] = df_LW_avg
    
    # save to the big dic
    data_dic[label]=data_lev_dic

# --- read GRU shapefile
gru_gpd = gpd.read_file(sub_shp_prj)
gru_gpd_prj = gru_gpd.to_crs(wgs_crs)

# --- read stream shapefile
stream_gpd = gpd.read_file(stream_clip)
stream_gpd_prj = stream_gpd.to_crs(wgs_crs)

# --- read HRU shapefile
hru_gpd_dic = {}
for i in range(level_num+1): # hru level
    label=label_basename+str(i) 

    # read HRU shapefile 
    hru_elmn_str = label+'_elmn'     
    hru_vector_elmn = os.path.join(case_dir, hru_elmn_str+'.shp')          
    hru_gpd = gpd.read_file(hru_vector_elmn)
    hru_gpd_prj = hru_gpd.to_crs(wgs_crs)
    hru_gpd_dic[label]=hru_gpd_prj

# ============== PART 2. Identify vmin and vmax ==============
print('vmin/vamx')
vmin_vmax_dic = {}
for i in range(level_num+1): # hru level
    label=label_basename+str(i) 
    vmin_vmax_lev_dic = {}
    
    for j in range(3): # variable
        var_name_summa = var_names_summa[j]

        # extract usefule variable value
        df = data_dic[label][var_name_summa]
        df_cut = df.truncate(before=plot_date_start_obj, after=plot_date_end_obj)
        data = df_cut.to_numpy()
        
        vmin,vmax = np.nanmin(data),np.nanmax(data)
        vmin_vmax_lev_dic[var_name_summa] = [vmin,vmax]
    
    # save to the big dic
    vmin_vmax_dic[label]=vmin_vmax_lev_dic    

# ============== PART 3. Daily plot ==============
print('plot')
pool = mp.Pool(mp.cpu_count())    
for t_obj in tqdm(plot_date_range[7:]):
    t_obj_str = t_obj.strftime('%Y-%m')
    
    nrow, ncol = level_num+1,3 # row: hru level. Col: shortwave rad, longwave rad, SWE
    fig, ax = plt.subplots(nrow, ncol,figsize=[7.08*1.5, 7.08*1.2], constrained_layout=True)
    fig.suptitle(t_obj_str, fontsize='medium',weight='semibold')

    for i in range(nrow): # hru level
        label=label_basename+str(i) 
        hru_gpd_prj = hru_gpd_dic[label]

        for j in range(ncol): # variable

            # identify variable and unit 
            var_name_summa = var_names_summa[j]
            var_name_plot = var_names_plot[j]
            var_unit_plot = var_units_plot[j]           
            vmin,vmax = vmin_vmax_dic[label][var_name_summa]

            # extract usefule variable value
            df = data_dic[label][var_name_summa] # dataframe (time, hru)
            df_cut = df.truncate(before=t_obj, after=t_obj)  
            data = df_cut.to_numpy()
            data = data.reshape((np.shape(data)[1],np.shape(data)[0]))            
            
            # plot
            # (1) plot gru
            gru_gpd_prj.geometry.boundary.plot(color=None,edgecolor='k',linewidth=0.5,
                                               ax=ax[i,j],label='GRU') 

#             # (2) plot stream
#             stream_gpd_prj.plot(color='b', linewidth=0.5, ax=ax[i,j], label='Stream')

            # (3) plot data
            # reference: https://geopandas.org/docs/user_guide/mapping.html
            # reference: https://www.martinalarcon.org/2018-12-31-d-geopandas/
            hru_gpd_prj[var_name_summa] = data
            divider = make_axes_locatable(ax[i,j])
            cax = divider.append_axes("right", size="5%", pad="0%")         
            hru_gpd_prj.plot(column=var_name_summa, ax=ax[i,j], cmap=cmaps[j], 
                             legend=True, cax=cax, vmin=vmin, vmax=vmax)
            
            # Set the fontsize for each colorbar tick label
            # manipulate the colorbar `cax`
            cax.set_title(var_unit_plot, fontsize='small')
            for l in cax.yaxis.get_ticklabels():
                l.set_fontsize('small')

            # others
            title = label+'\n'+var_name_plot
            ax[i,j].set_title(title,fontsize='small',weight='semibold')
            ax[i,j].tick_params(axis='both', direction='out', labelsize='small')
    
    ofile = t_obj.strftime('%Y%m')+'.png'
    plt.savefig(os.path.join(output_subdir,ofile), dpi=80)
    plt.close(fig)    
pool.close()   

## Then convert daily plots to gif using command line.
## convert -delay 240 plot_daily/*.png test.gif
print('Done')

read
vmin/vamx
plot


100%|██████████| 5/5 [02:07<00:00, 25.47s/it]

Done


In [22]:
plot_date_range

[datetime.datetime(2007, 10, 31, 0, 0),
 datetime.datetime(2007, 11, 30, 0, 0),
 datetime.datetime(2007, 12, 31, 0, 0),
 datetime.datetime(2008, 1, 31, 0, 0),
 datetime.datetime(2008, 2, 29, 0, 0),
 datetime.datetime(2008, 3, 31, 0, 0),
 datetime.datetime(2008, 4, 30, 0, 0),
 datetime.datetime(2008, 5, 31, 0, 0),
 datetime.datetime(2008, 6, 30, 0, 0),
 datetime.datetime(2008, 7, 31, 0, 0),
 datetime.datetime(2008, 8, 31, 0, 0),
 datetime.datetime(2008, 9, 30, 0, 0)]

In [9]:
df_sum.index

Index(['2007/10', '2007/11', '2007/12', '2008/01', '2008/02', '2008/03',
       '2008/04', '2008/05', '2008/06', '2008/07', '2008/08', '2008/09'],
      dtype='object')